In [11]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.types import *
from hit_rate import HitRate

import pandas as pd

In [5]:
spark = SparkSession.builder.master("local[2]") \
                    .appName('MovieRecomender') \
                    .getOrCreate()
print(spark.sparkContext)
print("Spark App Name : "+ spark.sparkContext.appName)

<SparkContext master=local[*] appName=MovieRecomender>
Spark App Name : MovieRecomender


In [6]:
import os 
os.chdir ('/home/mt/MovieRecomender')
#change to your working directory
%pwd

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/home/jovyan/work/MovieRecomender'

In [13]:
customSchema = StructType([
    StructField("UserID", LongType(), True), \
    StructField("MovieID", IntegerType(), True), \
    StructField("Rating", LongType(), True), \
    StructField("Timestamp", LongType(), True)
])

df = spark.read.option("sep", "::").option("encoding", 'ISO-8859-1').schema(customSchema).csv("data/ratings.dat")
df = df.cache() # Force cache
df.count()

In [15]:
(train, test) = df.randomSplit([0.8, 0.2])

In [16]:
als = ALS(userCol="UserID", itemCol="MovieID", ratingCol="Rating", nonnegative = True, implicitPrefs = False)

In [20]:
grid_search = ParamGridBuilder().addGrid(als.rank, [10, 25, 50 ,75 , 100]  ).addGrid(als.maxIter,[5, 10, 20, 40, 80, 160]  ).addGrid(als.regParam, [.001,.005,.01, .05, .1, .5] ).build()

RMSE

In [21]:
rmse_evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction") 

In [22]:
cv = CrossValidator(estimator=als, estimatorParamMaps=grid_search, evaluator=rmse_evaluator, numFolds=5)

22/10/04 23:02:53 WARN TaskSetManager: Stage 26 contains a task of very large size (7756 KiB). The maximum recommended task size is 1000 KiB.


22/10/04 23:03:07 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/04 23:03:07 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/10/04 23:03:10 WARN TaskSetManager: Stage 95 contains a task of very large size (7756 KiB). The maximum recommended task size is 1000 KiB.


In [23]:
cv_fitted=cv.fit(train)

KeyboardInterrupt: 

Hit Rate

In [18]:
hr_evaluator = HitRate(predictionCol='prediction', labelCol='rating', userCol='userId')

In [19]:
model = als.fit(train)
predictions = model.transform(test)
hr_evaluator.evaluate(predictions)

+------+-----+
|userId|count|
+------+-----+
|     1|    8|
|     5|   10|
|     6|   10|
|     7|    7|
|     9|   10|
|    10|   10|
|    17|   10|
|    19|   10|
|    22|   10|
|    25|   10|
|    26|   10|
|    27|   10|
|    28|   10|
|    29|   10|
|    31|   10|
|    32|    3|
|    33|   10|
|    34|   10|
|    39|   10|
|    41|    4|
+------+-----+
only showing top 20 rows



1.0